# Prim algorithm with heap data structure 

In [1016]:
import math
import time
from collections import defaultdict

## MinHeap data structure implementation

In [1017]:
# ArrayHeap object extends list
class ArrayHeap(list):
    def __init__(self, array):
        super().__init__(array)
        #self.heapSize = 0
        self.heapSize = len(array)

class MinHeap:
    def __init__(self, array):
        self.arrayHeap = ArrayHeap(array)
    
    # All the following methods work with zero based array.
    def parent(self, i):
        if i == 2: 
            return 0
        else:
            return math.floor(i/2)
    
    def left(self, i):
        return 2*i + 1
    
    def right(self, i):
        return 2*i + 2
    
    # Execution time: O(lg n)
    def minHeapify(self, i):
        l = self.left(i)
        r = self.right(i)
        if l <= self.arrayHeap.heapSize-1 and self.arrayHeap[l].key < self.arrayHeap[i].key:
            minimo = l
        else:
            minimo = i
        if r <= self.arrayHeap.heapSize-1 and self.arrayHeap[r].key < self.arrayHeap[minimo].key:
            minimo = r
        if minimo != i:
            self.arrayHeap[i], self.arrayHeap[minimo] = self.arrayHeap[minimo], self.arrayHeap[i]
            self.minHeapify(minimo)
            
    # Execution time: O(n)
    def buildMinHeap(self):
        #self.arrayHeap.heapSize = len(self.arrayHeap)
        for i in range(math.floor(len(self.arrayHeap)/2), -1, -1): #downto
            self.minHeapify(i)

    def bubbleUp(self, index):
        parent = self.parent(index)
        current = index
        while current > 0 and self.arrayHeap[parent].key > self.arrayHeap[current].key:
            self.arrayHeap[current], self.arrayHeap[parent] = self.arrayHeap[parent], self.arrayHeap[current]
            current = parent
            parent = self.parent(parent)
    
    def decreaseKey(self, destIndex, destination, key):
        destination.key = key
        self.bubbleUp(destIndex)
    
    # Execution time: O(lg n)
    # First we update the heap structure, then we remove the last element.
    def extractMin(self):
        if self.arrayHeap.heapSize < 0:
            print("Error: extractMin underflow")
            return
        else:
            minimo = self.arrayHeap[0]
            print("node=", minimo.tag,"minimo=",minimo.key)
            self.arrayHeap[0].isPresent = False

            self.arrayHeap[0] = self.arrayHeap[self.arrayHeap.heapSize - 1]
            # This action is needed in order to remove the last element of the list. Otherwise, we will obtain
            # a duplicate of the last element.
            self.arrayHeap.pop(self.arrayHeap.heapSize-1)
            self.arrayHeap.heapSize -= 1
            self.minHeapify(0)

            return minimo

## Definition of Node and Graph objects for graph manipluation        

In [1018]:
class Node:
    def __init__(self, tag):
        self.tag = tag
        self.key = None
        self.parent = None
        self.isPresent = True
        self.adjacencyList = []
        
    # For test 
    def print(self):
        print("tag =", self.tag, "adjList=", self.adjacencyList, "key=", self.key)

class Graph:
    def __init__(self):
        self.nodes = defaultdict(Node)
        self.cut = [] #For test, used to save cut nodes
        
    def createNode(self, nums):
        for i in range(1, nums+1): # nums+1 in order to cover the last node
            self.nodes[i] = Node(i)

    # If two nodes have more than one connection, it saves only the cheaper
    def addNode(self, tag, adjList):
        adjTag = adjList[0]
        adjCost = adjList[1]
        self.nodes[tag].adjacencyList.append([self.nodes[adjTag], adjCost])
        self.nodes[adjTag].adjacencyList.append([self.nodes[tag], adjCost])

    def buildGraph(self, input):
        lines = input.readlines()
        self.createNode(int(lines[0].split()[0])) # Extract number of vertexes and pass it to createNode
        lines.pop(0) 
        for line in lines:
            info = list(map(int, line.split())) # Convert all the strings deriving from split to int
            self.addNode(info[0], [info[1], info[2]])

## Prim algorithm

In [1019]:
def MSTPrim(g, r):
    for node in g.nodes.values():
        node.key = math.inf # Set key while parent is already set
    r.key = 0
    g.cut.append(r)
    q = MinHeap(list(g.nodes.values()))
    q.buildMinHeap() # This function call is necessary in order to call MSTPrim with any root node
    while len(q.arrayHeap) is not 0:
        for node in q.arrayHeap:
            print(node.tag, node.key)
        #for node in g.cut:
        #    print(node.key, node.isPresent)
        u = q.extractMin()
        #g.cut.remove(u)
        #print("extract =",u.tag,"with key =", u.key)
        for v in u.adjacencyList:
            #if v[0].isPresent and v[1] < v[0].key:
            #    v[0].parent = u
            #    v[0].key = v[1]
            #    q.bubbleUp(q.arrayHeap.index(v[0]))
            #    print(v[0].tag, v[0].key)
            if v[0].isPresent:
                #g.cut.append(v[0])
                if v[1] < v[0].key:
                    v[0].parent = u
                    v[0].key = v[1]
                    q.bubbleUp(q.arrayHeap.index(v[0]))
                    #print(v[0].tag, v[0].key)
        #print("\n")

In [1020]:
inp = open("dataset/input_random_01_10.txt", "r")
lines = inp.readlines()
start = time.time()
result = Graph()
result.buildGraph(open("dataset/input_random_01_10.txt", "r"))
MSTPrim(result, result.nodes.get(2))
print("partenza nodo",2)
print("Execution time =", time.time() - start)
sum = 0
for node in result.nodes.values():
    #print(node.tag, node.key)
    sum += node.key
print(sum, "\n")
#for i in range(1,int(lines[0].split()[0])+1, 1):
#    result = Graph()
#    result.buildGraph(open("dataset/input_random_01_10.txt", "r"))
#    MSTPrim(result, result.nodes.get(i))
#    print("partenza nodo",i)
#    print("Execution time =", time.time() - start)
#    sum = 0
#    for node in result.nodes.values():
        #print(node.tag, node.key)
#        sum += node.key
#    print(sum, "\n")

2 0
1 inf
3 inf
4 inf
5 inf
6 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
16 inf
17 inf
18 inf
19 inf
20 inf
node= 2 minimo= 0
3 -1086
20 inf
1 3443
4 inf
5 inf
6 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
16 inf
17 inf
18 inf
19 inf
node= 3 minimo= -1086
4 100
1 3443
19 8619
20 inf
5 inf
6 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
16 inf
17 inf
18 inf
node= 4 minimo= 100
5 -4993
18 inf
1 3443
20 inf
19 8619
6 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
16 inf
17 inf
node= 5 minimo= -4993
1 3443
18 inf
6 8307
20 inf
19 8619
17 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
16 inf
node= 1 minimo= 3443
6 8307
18 inf
16 inf
20 inf
19 8619
17 inf
7 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
15 inf
node= 6 minimo= 8307
7 -7057
15 inf
16 inf
18 inf
19 8619
17 inf
20 inf
8 inf
9 inf
10 inf
11 inf
12 inf
13 inf
14 inf
node= 7 minimo= -7057
8 9698
14 inf
16 inf
15 inf
19 8619
17 inf
20 in